In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

  # Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive


In [ ]:
from pathlib import Path
# wandb library
import wandb
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn import metrics
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed

import datasets
from datasets import Dataset, DatasetDict

import evaluate

import torch

!git lfs install

Git LFS initialized.


In [ ]:
base_folder = Path(basepath)
data_folder = base_folder/'Homework/Homework7'
model_folder = base_folder/'models/nlp_spring_2024/ed/nn'
custom_functions = base_folder/'custom-functions'

In [ ]:
model_folder.mkdir(exist_ok=True, parents = True)

In [ ]:
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')
import CustomPreprocessorSpacy as cp

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
train_data = pd.read_csv(data_folder/ 'train.csv')
test= pd.read_csv(data_folder/ 'test.csv')

In [ ]:
train_data.drop(columns=["ID"],inplace=True)
y_test = test.drop(['ID','Tweet'],axis=1)
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(int, x)))
test_data = pd.concat([test['Tweet'],y_test],axis=1)

train_data.head()


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [ ]:
test_data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,0,0,0,0,0,0,0,0,0,0,0
1,"Academy of Sciences, eschews the normally sobe...",0,0,0,0,0,0,0,0,0,0,0
2,I blew that opportunity -__- #mad,0,0,0,0,0,0,0,0,0,0,0
3,This time in 2 weeks I will be 30... 😥,0,0,0,0,0,0,0,0,0,0,0
4,#Deppression is real. Partners w/ #depressed p...,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
labels = [x for x in train_data.columns if x not in ['Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
NUM_OF_LABELS = len(labels)

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_testEval = train_dataset.train_test_split(train_size=0.80)
val_eval = train_testEval['test']

ds = DatasetDict({
    'train' : train_testEval['train'],
    'val' : val_eval,
    'test':test_dataset

})

print("Training Dataset Shape:", ds['train'].shape)
print("Evaluation Dataset Shape:", ds['val'].shape)
print("Testing Dataset Shape",ds['test'].shape)
ds['train'][25]

Training Dataset Shape: (6179, 12)
Evaluation Dataset Shape: (1545, 12)
Testing Dataset Shape (3259, 12)


{'Tweet': 'I should have stayed at home.',
 'anger': 0,
 'anticipation': 0,
 'disgust': 1,
 'fear': 0,
 'joy': 0,
 'love': 0,
 'optimism': 0,
 'pessimism': 0,
 'sadness': 0,
 'surprise': 0,
 'trust': 0}

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 6179
    })
    val: Dataset({
        features: ['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 1545
    })
    test: Dataset({
        features: ['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 3259
    })
})

In [ ]:
set_seed = 42
NUM_OF_EPOCHS = 10

BATCH_SIZE = 128
LEARNING_RATE = 1e-4

MAX_LENGTH = 128
MODEL_CKPT = "distilroberta-base"
MODEL_NAME = MODEL_CKPT + "-reviews_multilabel_clf_v2"

METRIC_NAME = "f1_macro"
DEVICE = torch.device("cpu")
print(f"This project will utilize the {DEVICE} to train and evaluate.")
print(f"This project will have a maximum length for the input text of {MAX_LENGTH} tokens.")

This project will utilize the cpu to train and evaluate.
This project will have a maximum length for the input text of 128 tokens.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def data_preprocessor(examples):
	# take a batch of text
	text = examples["Tweet"]
	# encode them
	encoded_data = tokenizer(text, padding="max_length", truncation=True, max_length=MAX_LENGTH)
	# add labels
	labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
	# create numpy array of shape (batch_size, num_labels)
	labels_matrix = np.zeros((len(text), len(labels)))
	# fill numpy array
	for idx, label in enumerate(labels):
		labels_matrix[:, idx] = labels_batch[label]

	encoded_data["labels"] = labels_matrix.tolist()

	return encoded_data

In [ ]:
encoded_ds = ds.map(data_preprocessor, batched=True, load_from_cache_file=False)
encoded_ds.set_format("torch")

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
print(f"The shape of the training dataset is: {encoded_ds['train'].shape}")

print(f"The shape of the evaluation dataset is: {encoded_ds['val'].shape}")
print(f"The shape of the test dataset is: {encoded_ds['test'].shape}")

The shape of the training dataset is: (6179, 15)
The shape of the evaluation dataset is: (1545, 15)
The shape of the test dataset is: (3259, 15)


In [ ]:
"""model = (RobertaForSequenceClassification.from_pretrained(MODEL_CKPT,
                                                             num_labels=NUM_OF_LABELS,
                                                             problem_type="multi_label_classification",
                                                             id2label=id2label,
                                                             label2id=label2id)
         ).to(DEVICE)"""

model = RobertaForSequenceClassification.from_pretrained(MODEL_CKPT,
                                                         num_labels=NUM_OF_LABELS,
                                                         problem_type="multi_label_classification",
                                                         id2label=id2label,
                                                         label2id=label2id)

# Add dropout layers
model.dropout1 = torch.nn.Dropout(p=0.2)
model.dropout2 = torch.nn.Dropout(p=0.2)
model.classifier.dropout = torch.nn.Dropout(p=0.2)

model = model.to(DEVICE)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def multi_label_metrics(predictions,
                        labels,
                        threshold=0.5):
    '''
    This function calculates & returns metrics
    for a multilabel classification analysis.
    '''

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true,y_pred=y_pred,average ='macro')

    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return metrics as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro':f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds = preds[0] if isinstance(preds,
                    tuple) else preds
    results = multi_label_metrics(
                    predictions=preds,
                    labels=labels)
    return results

In [ ]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    lr_scheduler_type='linear',
    num_train_epochs=NUM_OF_EPOCHS,
    optim='adamw_torch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    warmup_steps = 0,
    greater_is_better=True,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_first_step=True,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Tweet_Emotion',
)

In [ ]:
trainer = Trainer(
    model,
    args,

    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
trainer.train()

wandb: Currently logged in as: rangareddy-bhargav. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Roc Auc,Accuracy
1,0.700900,0.367752,0.595493,0.356414,0.729467,0.190939
2,0.700900,0.323255,0.646226,0.439699,0.756471,0.252427
3,0.700900,0.314809,0.673406,0.508342,0.779256,0.254369
4,0.700900,0.314267,0.685797,0.550017,0.790347,0.260841
5,0.700900,0.318674,0.682469,0.557902,0.788764,0.258252
6,0.700900,0.319756,0.684951,0.554722,0.789640,0.261489
7,0.700900,0.329460,0.676344,0.555386,0.781489,0.269256
8,0.700900,0.334204,0.678342,0.562083,0.784837,0.260841
9,0.700900,0.339612,0.682871,0.564947,0.789751,0.260841
10,0.700900,0.341414,0.681864,0.567381,0.788662,0.259547


TrainOutput(global_step=490, training_loss=0.2407189995658641, metrics={'train_runtime': 716.7478, 'train_samples_per_second': 86.209, 'train_steps_per_second': 0.684, 'total_flos': 2046618574855680.0, 'train_loss': 0.2407189995658641, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3414139151573181,
 'eval_f1_micro': 0.6818640888760641,
 'eval_f1_macro': 0.5673807386098111,
 'eval_roc_auc': 0.7886620533891283,
 'eval_accuracy': 0.25954692556634307,
 'eval_runtime': 5.3134,
 'eval_samples_per_second': 290.774,
 'eval_steps_per_second': 2.447,
 'epoch': 10.0}

In [ ]:
valid_output = trainer.predict(encoded_ds["val"])

In [ ]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [ ]:
valid_preds = np.argmax(valid_output.predictions, axis=-1)
valid_labels = np.array(valid_output.label_ids)


In [ ]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 490.


In [ ]:
model_checkpoint = str(model_folder) + '/checkpoint-' + best_model_checkpoint_step

In [ ]:
valid_results = trainer.evaluate(encoded_ds["val"])

In [ ]:
valid_results

{'eval_loss': 0.3414139151573181,
 'eval_f1_micro': 0.6818640888760641,
 'eval_f1_macro': 0.5673807386098111,
 'eval_roc_auc': 0.7886620533891283,
 'eval_accuracy': 0.25954692556634307,
 'eval_runtime': 5.2874,
 'eval_samples_per_second': 292.205,
 'eval_steps_per_second': 2.459,
 'epoch': 10.0}

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint)
model.to('cuda')
model.eval()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_ds['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
test_results = trainer.evaluate()

In [ ]:
wandb.finish()

eval/accuracy,▁▆▇▇▇▇█▇▇▇▇▇
eval/f1_macro,▁▄▆▇████████
eval/f1_micro,▁▅▇███▇▇████
eval/loss,█▂▁▁▂▂▃▄▄▅▅▅
eval/roc_auc,▁▄▇███▇▇████
eval/runtime,▁▄▂▅▂█▅▃▄▇▅▃
eval/samples_per_second,█▅▇▄▇▁▄▆▅▂▄▆
eval/steps_per_second,█▅▆▄▇▁▄▆▅▂▄▆
train/epoch,▁▂▂▃▄▄▅▆▇▇████
train/global_step,▁▂▂▃▄▄▅▆▇▇████
train/grad_norm,▁
